# 1. 라이브러리 선언

In [1]:
import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# 2. 이미지 전처리 수행

In [2]:
# 인풋 이미지를 불러와서 (28,28,1 형태로 변경, 합성곱 신경망 활용)
def process(img_input):
    # 이미지 사이즈 변경
    IMG_SIZE = 28
    # 그레이컬러 변환 및 사이즈 조절 (단 이미 gray인 경우 패스)
    if len(testimg.shape) > 2:
        img_input = cv2.cvtColor(img_input, cv2.COLOR_BGR2GRAY)
    img_input = cv2.resize(img_input, (IMG_SIZE, IMG_SIZE))
    # 합성곱 신경망 적용을 위한 설정
    out_img = img_input.reshape(1,IMG_SIZE,IMG_SIZE,1)
    return out_img

# 3. 모델 불러오기

In [3]:
import json
from tensorflow.keras.models import model_from_json

with open('model_mnist.json', 'r') as json_file:
    loaded_model = model_from_json(json_file.read())

loaded_model.summary()
loaded_model.load_weights("./model_mnist_weight.h5")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1568)              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 7

In [4]:
import cv2

testimg = cv2.imread(\
     "./sample_mnist.jpg", cv2.IMREAD_GRAYSCALE)

In [5]:
prepro_img = process(testimg)
prepro_img.shape

(1, 28, 28, 1)

In [6]:
predictValue = loaded_model.predict(prepro_img)
np.argmax(predictValue)

3

# 4. 웹캠 연동

In [7]:
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
CAP_SIZE = 150
while(True):
    ret, img_color = cap.read()
    if ret == False:
        break;
    x1= int(width/2-CAP_SIZE/2)
    y1=int(height/2-CAP_SIZE/2)
    x2= x1+CAP_SIZE
    y2= y1+CAP_SIZE
    cv2.rectangle(img_color, ( x1,y1 ), ( x2,y2 ),  (0, 0, 255), 3)  
    cv2.imshow('bgr', img_color)

    img_roi = img_color[y1:y1+CAP_SIZE, x1:x1+CAP_SIZE]
    key = cv2.waitKey(33)
    # 코드 Here
    if key==27: # esc key
        cap.release()
        cv2.destroyAllWindows()
    elif key==32: # space bar key
        try:
            target_img = process(img_roi)
            p_value = loaded_model.predict(target_img)
            p_value2 = np.argmax(p_value)
            print(p_value2)
            cv2.imwrite("./test_capture.jpg", img_roi)
        except Exception as e:
            print(e)

cap.release()
cv2.destroyAllWindows()

cannot reshape array of size 2352 into shape (1,28,28,1)
cannot reshape array of size 2352 into shape (1,28,28,1)


In [11]:
cv2.destroyAllWindows()

In [28]:
test = cv2.imread("../images/mnist/trainingSet/2_two/img_10247.jpg", cv2.IMREAD_COLOR)

# test = cv2.imread("d:/aaa.jpg", cv2.IMREAD_COLOR)

test2 = process(test)

test3 = test2.reshape(28,28)*255

plt.imshow(test3, cmap="gray")

np.argmax(loaded_model.predict(test2.reshape(1,28,28,1)))

In [8]:
from tensorflow.keras import backend as K

K.clear_session()

from numba import cuda
cuda.select_device(0)
cuda.close()